In [4]:
from elasticsearch import Elasticsearch

In [7]:
from tqdm.auto import tqdm

In [8]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [11]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
es_client = Elasticsearch('http://localhost:9200') 

In [12]:
es_client.info()

ObjectApiResponse({'name': '1a3e11613766', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'LvYsC4uxQtSM44Sw496T2w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [17]:
# Ensure the index is created with the given settings and mappings
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

# Delete the index if it already exists (optional, for clean slate)
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)

# Create the index
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [18]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [14]:
query = 'How do I execute a command in a running docker container?'

In [78]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        section = doc['_source'].get('section', 'N/A')
        question = doc['_source'].get('question', 'N/A')
        answer = doc['_source'].get('text', 'N/A')
        context += f"section: {section}\nquestion: {question}\nanswer: {answer}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [79]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    # Adding a predefined Docker question and answer
    docker_context = """
Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
"""

    context = ""
    
    for doc in search_results:
        section = doc['_source'].get('section', 'N/A')
        question = doc['_source'].get('question', 'N/A')
        answer = doc['_source'].get('text', 'N/A')
        context += f"section: {section}\nquestion: {question}\nanswer: {answer}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


context_template = """
Q: {How do I copy files from my local machine to docker container?}
A: {You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it: docker cp /path/to/local/file_or_directory container_id:/path/in/container}
""".strip()

In [80]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

client = MistralClient()


def llm(prompt):
    response = client.chat(
        model='open-mistral-7b',
        messages=[
            ChatMessage(role="user", content=prompt)
        ]
    )

    return response.choices[0].message.content

In [81]:
query1= "What's the 3rd question returned by the search engine?"

In [90]:
query2="How do I execute a command in a running docker container?"

In [91]:
def elastic_search(query2):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
         result_docs.append({
            "_score": hit['_score'],
            "_source": hit['_source']})
        
    
    return result_docs

In [92]:
def rag(query2):
    search_results = elastic_search(query2)
    prompt = build_prompt_context(query2, search_results)
    answer = llm(prompt)
    return answer

In [95]:
len(rag(query2))

513

In [96]:
query = "How do I execute a command in a running docker container?"
results = elastic_search(query)
for result in results:
    print(f"Score: {result['_score']}")
    print()

Score: 75.54128

Score: 43.922554

Score: 38.684105

Score: 38.33403

Score: 35.94081



In [118]:
import tiktoken

# Get the encoding for a specific model in the OpenAI API
enc = tiktoken.encoding_for_model("gpt-4o")


# Encode the prompt
tokens = enc.encode(rag(query2))

# Calculate the number of tokens
num_tokens = len(tokens)

# Print the number of tokens
print("Number of tokens:", num_tokens)


Number of tokens: 345
